In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *

from tqdm.notebook import tqdm

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

In [3]:
load_from_disk = True

# '2Min'

In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [5]:

def getModel():
    param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
    #param = {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 3, 'random_state': 42}
    coModel = RandomForestClassifier(**param)
    return coModel

In [6]:
%%time

def get_classifiers():
    clss = classifiers = [
        #lambda : make_pipeline(StandardScaler(),
        #                        MLPClassifier(
        #                            solver='lbfgs', alpha=1, random_state=1, max_iter=20000,
        #                            early_stopping=True)
        #                        ),
        
        #lambda : make_pipeline(StandardScaler(),
        #                        MLPClassifier(alpha=1, random_state=1, max_iter=20000,
        #                            early_stopping=True)
        #                        ),
        
        lambda : xgb.XGBClassifier(random_state=1,
                                   objective = "binary:logistic",
                                   eval_metric='mlogloss',
                                   learning_rate=0.01),
        
        #lambda : GaussianNB(),
        #lambda : QuadraticDiscriminantAnalysis(),
        #lambda : AdaBoostClassifier(random_state = 42),
        lambda : CatBoostClassifier(logging_level = 'Silent'),
        
        
        #lambda : KNeighborsClassifier(3),
        #lambda : GaussianProcessClassifier(1.0 * RBF(1.0)),
        
        
        #lambda : DecisionTreeClassifier(random_state = 42),
        #lambda : RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
        
        #lambda : SVC(gamma=2, C=1, random_state = 42),
    ]
    
    def rand():
        ##print("rand created")
        param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
        return RandomForestClassifier(**param)
    

    #clss = []
    clss.append(lambda : rand())
    return clss


estimators = [
                lambda : RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10),
                lambda : RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=8),
                lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10)),
                lambda : RFECV(estimator=RandomForestClassifier(random_state=10))
                ]


scoring = [
    #'accuracy', 
    #'f1', 
    #'precision', 
    'recall'
]

estimators = []

for score in scoring:
    
    est = lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                        scoring=score)
    
    estimators.append(est)


def get_all_models():
    models = []
    classifiers = get_classifiers()
    for cls in classifiers:
        models.append(cls())
        #models.append(MockModel(cls()))
        #models.append(MockCoPilotModel(cls(), getModel()))
        
        for est in estimators:
            tmp = lambda : Pipeline(
                steps=[
                ('s',est()),
                ('m',cls())
            ])
            models.append(tmp())
            #models.append(MockModel(tmp()))
            #models.append(MockCoPilotModel(tmp(), getModel()))
    return models


CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 29.1 µs


In [7]:
get_all_models_factory = lambda : get_all_models()

In [8]:
%%time

def test_models(provider, get_all_models_factory, steps = [1]):
    
    score_board_by_step = {}

    for step in steps:
        print(f"Step {step}")
        models_list = get_all_models_factory()
        model_result = []
        for idx in tqdm(range(len(models_list))):
            model_creator = models_list[idx]
            model = model_creator
            #print(f"Trainning {model}")
            train_by_step(model, step, provider)
            
            result = run_trial(model, provider, step)

            result['step'] = step

            model_result.append(result)
        
        score_board_by_step[step] = model_result
        
    best_models = []

    best_model = []

    best_score = -1
    
    best_step = -1
    
    model_rank = []
    
    def myFunc(e):
        return e['profit']

    for step_idx in score_board_by_step:
        step_board = score_board_by_step[step_idx]
        for result in step_board:
            if (result['profit'] > 0):
                model_rank.append(result)
            
    model_rank.sort(key=myFunc, reverse = True)
    
    return model_rank

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.72 µs


In [9]:

steps = [1, 3]

windows = [5, 10, 20, 30, 40]
#windows = [5, 10, 40]

minutes = [15, 5, 3, 1]
#minutes = [1]


In [10]:

def test_windows(minutes, steps = steps):
    results = []
    for win in tqdm(windows):
        tec = TecAn(windows = win, windows_limit = 100)
        sourceDataGenerator = SourceDataGenerator(tec = tec)
        online = get_online_data(minutes, sourceDataGenerator, load_from_disk, win)
        result = test_models(
            online, 
            get_all_models_factory,
            steps = steps
        )
        for rank in result:
            rank['window'] = win
            rank['minutes'] = minutes
            
        results.append(result)
    return results
    

In [10]:
%%time

min_results = []

for minu in tqdm(minutes):
    print(f"Evaluating {minu} min")
    results = test_windows(minu, steps)
    min_results.append(results)


  0%|          | 0/4 [00:00<?, ?it/s]

Evaluating 15 min


  0%|          | 0/5 [00:00<?, ?it/s]

Step 1


  0%|          | 0/6 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/magics/execution.py", line 1321, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 5, in <module>
  File "<ipython-input-10-406e83f5dcdf>", line 7, in test_windows
    result = test_models(
  File "<timed exec>", line 13, in test_models
  File "/Users/samirantonio/Desktop/YaStockRnn/backtest.py", line 122, in train_by_step
    model.fit(x, y)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/User

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/magics/execution.py", line 1321, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 5, in <module>
  File "<ipython-input-10-406e83f5dcdf>", line 7, in test_windows
    result = test_models(
  File "<timed exec>", line 13, in test_models
  File "/Users/samirantonio/Desktop/YaStockRnn/backtest.py", line 122, in train_by_step
    model.fit(x, y)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/User

TypeError: object of type 'NoneType' has no len()

In [ ]:
model_path = f'model/all_results'
dump(min_results, model_path)

In [ ]:
best_results = []
for results in min_results:
    for r in results:
        if (len(r) > 0):
            for win_result in r:
                profit = win_result['profit']
                model = win_result['model']
                window = win_result['window']
                minutes = win_result['minutes']
                step = win_result['step']
                profit = win_result['profit']
                models_score = win_result['models_score']
                #print(models_score)
                if (profit > 100 and minutes == 1):
                    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
                    print(f"{win_result['model']}")
                    print(f"")
                    best_results.append(win_result)
        else:
            print("Not found")

print(f"{len(best_results)} selected")

In [ ]:
model_path = f'model/results'
dump(best_results, model_path)

In [ ]:
def print_result(result):
    model = result['model']
    window = result['window']
    minutes = result['minutes']
    step = result['step']
    profit = result['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit} {best['models_profit_metric']}")
    print(f"{model}")

In [ ]:
def myFunc(e):
    return e['profit']
    #return e['models_profit_metric']['btcusd']

best_results.sort(key=myFunc, reverse = False)

for best in best_results:
    print_result(best)
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    model_path = f'model/result_{window}_{minutes}_{step}'
    dump(best, model_path)
    print(f"")

In [ ]:
def myFunc(e):
    #return e['profit']
    return e['models_profit_metric']['btcusd']

best_results.sort(key=myFunc, reverse = True)

In [ ]:
best_results[0]

In [ ]:
def load_online(minutes, window, val_end = 1627061284, currency = []):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)


    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 40,
                 val_limit = 999,
                 val_keys = currency,
                 val_start = 1627035284,
                 val_end = val_end,
                 train_start_list = []
    )

    online.load_cache()
    return online

In [ ]:
def backtest_model(model, x, closed_prices, back):
    
    for idx in range(len(x)):
        xx = np.array([x[idx]])
        #print(xx)
        yy = model.predict(xx)
        yy = yy[0]
        price = closed_prices[idx]
        #print(f'{idx} {yy} {price}')
        back.on_state(0, price)
        if(yy == 1):
            back.request_buy(price)
        else:
            back.request_sell(price)
    
    #print(f"Closing backtest_model {price} {back.current}")
    back.sell(price)
    #print(f"Closed backtest_model {back.current}")
    
    return back

In [ ]:
def test_model(model, set_key, provider, step, verbose = True):
    valX, valY = provider.load_val_data(set_key)

    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    #print(len(x))
    preds = model.predict(x)

    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    f1 = f1_score(y, preds)
    accuracy = accuracy_score(y, preds)

    back = BackTest(value = 100, 
                    verbose = verbose, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    back = backtest_model(model, x, closed_prices, back)
    return back

In [ ]:
from datetime import datetime


timestamp = int(datetime.timestamp((datetime.now())))


timestamp = 1627061284

In [ ]:
#online_cache = {}

In [ ]:
scoreboard = []

fails = []

currency = "ltcusd"
currency = "ethusd"
currency = "batusd"

#currs = ["batusd", "ltcusd", "ethusd", "btcusd"]
currs = ["btcusd"]

#currs = online.val_keys

for best in tqdm(best_results):
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    profit = best['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
    cache_key = f"{minutes}-{window}"
    try:
        online = online_cache[cache_key]
    except :
        print(f"Not found {cache_key}")
        online = load_online(minutes = minutes, window = window, val_end = timestamp, currency = currs)
        online_cache[cache_key] = online
        
    profits = []
    backs = {}
    score = {}
    
    has_negative = False
    
    for train_set in online.val_keys:
        back = test_model(model, train_set, online, step, False)
        back_profit = back.get_profit()
        profits.append(back_profit)
        backs[train_set] = back
        if (back_profit <= 0 and not has_negative):
            has_negative = True
        
    score['profit'] = np.average(profits)
    score['backs'] = backs
    score['result'] = best
    if (not has_negative):
        scoreboard.append(score)

    


In [ ]:
profs = []
for score in scoreboard:
    profit = score['profit']
    result = score['result']
    backs = score['backs']
    print(f"{profit}")
    for back in backs:
        print(f"{back} -> {backs[back].get_profit()}")
    print_result(result)
    #print(result)
    profs.append(profit)
    model_path = f'model/winner'
    dump(result, model_path)

    
print()
print(f"{np.average(profs)} ")

In [ ]:
model = scoreboard[0]['result']['model']

In [ ]:
model